# Word2Vec from scratch

## 16. Explain how Word2Vec learns? What is the loss function? What is maximized?


## build word2vec from scratch

Word2Vec est un modèle de type embeddings qui représente les mots sous forme de vecteurs denses dans un espace de dimension réduite, de manière à capturer le contexte d'un mot dans une phrase.

il apprend de deux manière
- Skip-Gram : Pour chaque mot cible, prédire les mots de contexte autour de lui.
- CBOW (Continuous Bag of Words) : Pour chaque ensemble de mots de contexte, prédire le mot cible.

- Utilisez Skip-gram si vous avez un petit corpus ou des mots rares à traiter.
- Utilisez CBOW si vous travaillez avec un grand corpus et un vocabulaire plus fréquent.

## Word2vec from scratch with Skip-Gram

<div align="center">
<img src= "img/word2vec.png"/>
<div>

In [11]:
import random
import numpy as np

# Exemple de corpus
corpus = [
    "le chat mange une pomme",
    "le chien dort sous l'arbre",
    "le chat dort aussi sous l'arbre"
]

# Tokenisation du corpus
tokens = [sentence.split() for sentence in corpus]
print(tokens)

# Création d'un dictionnaire de vocabulaire
vocab = set(word for sentence in tokens for word in sentence)
word_to_index = {word: i for i, word in enumerate(vocab)}

# Fonction pour extraire les paires contexte-cible (Skip-Gram)
def generate_pairs(tokens, window_size=1):
    pairs = []
    for sentence in tokens:
        for i, word in enumerate(sentence):
            target = word_to_index[word]  # Cible
            context_words = [sentence[j] for j in range(i - window_size, i + window_size ) if j != i]
            for context in context_words:
                pairs.append((target, word_to_index[context]))  # (cible, contexte)
    return pairs

# Génération des paires contexte-cible
pairs = generate_pairs(tokens, window_size=1)
vocab_list = list(vocab)

[['le', 'chat', 'mange', 'une', 'pomme'], ['le', 'chien', 'dort', 'sous', "l'arbre"], ['le', 'chat', 'dort', 'aussi', 'sous', "l'arbre"]]


In [13]:
# Initialisation aléatoire des vecteurs de mots
vocab_size = len(vocab)
embedding_dim = 10  # Taille des vecteurs d'embedding

# Matrices de poids : taille (vocab_size, embedding_dim)
W1 = np.random.rand(vocab_size, embedding_dim)  # Poids pour les cibles
W2 = np.random.rand(vocab_size, embedding_dim)  # Poids pour les contextes

In [14]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Fonction de perte : log-likelihood négatif
def loss_function(target, context, W1, W2):
    score = np.dot(W1[target], W2[context])  # Produit scalaire
    return -np.log(sigmoid(score))

# Gradient de la perte par rapport aux poids
def gradient(target, context, W1, W2, learning_rate=0.1):
    score = np.dot(W1[target], W2[context])
    prediction = sigmoid(score)
    error = prediction - 1  # L'erreur pour une paire (cible, contexte) positive
    grad_W1 = error * W2[context]  # Gradients pour W1 (cibles)
    grad_W2 = error * W1[target]  # Gradients pour W2 (contextes)
    return grad_W1, grad_W2

# Entraînement : descente de gradient
for epoch in range(100):  # Nombre d'itérations
    total_loss = 0
    for target, context in pairs:
        grad_W1, grad_W2 = gradient(target, context, W1, W2)
        W1[target] -= 0.1 * grad_W1  # Mise à jour des poids
        W2[context] -= 0.1 * grad_W2
        total_loss += loss_function(target, context, W1, W2)
    
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {total_loss}")


Epoch 0, Loss: 0.9964132099431408
Epoch 10, Loss: 0.5400119713758561
Epoch 20, Loss: 0.36222910801582203
Epoch 30, Loss: 0.26942969293131896
Epoch 40, Loss: 0.2129356807600291
Epoch 50, Loss: 0.17513571931186578
Epoch 60, Loss: 0.14817262575221285
Epoch 70, Loss: 0.12802918546287062
Epoch 80, Loss: 0.11244433699353756
Epoch 90, Loss: 0.1000510165625448


In [ ]:
# Vérification des vecteurs de mots
for word, index in word_to_index.items():
    print(f"{word}: {W1[index]}")

le: [0.80665593 0.33176658 1.05986271 0.81997102 0.57681536 1.21649046
 0.88467455 0.87910907 1.10978974 0.71079258]
sous: [0.31387691 0.78417543 0.82889312 0.27291319 0.74923362 1.08794956
 1.01249055 1.10678309 0.91419246 0.53670476]
pomme: [0.44766915 0.72413787 0.48041782 0.30444487 0.99350513 0.15005236
 0.52801828 0.93741134 0.28213302 1.057748  ]
dort: [0.69846803 0.95090972 0.8930665  0.32154198 0.63480515 0.7621131
 0.86595167 0.66995676 1.04502992 1.13044216]
chat: [0.64022845 0.24352789 0.74688968 0.61558872 0.27848528 0.72043005
 1.16949664 0.28555824 0.47269413 0.79113151]
une: [1.06315688 0.34945953 0.340403   0.40011085 0.87832048 1.01584965
 0.44408634 0.6665143  0.26676153 0.96386479]
chien: [0.73238471 0.51687325 0.67419172 0.79377774 0.33902238 0.45904576
 0.72771295 0.55566983 0.71006662 0.38612705]
mange: [0.92426893 0.96817434 0.32938624 0.10810074 0.21039087 0.3170874
 1.21621041 1.02209608 0.10278668 0.67942506]
l'arbre: [0.71149297 0.64656014 0.23157997 0.13588

## 18. What is the difference between static and contextual embeddings?


### **Static Embeddings (Représentations statiques)** :
- **Vecteur fixe** : Chaque mot a un vecteur unique, quelle que soit sa signification ou son contexte.
- **Exemples** : Word2Vec, GloVe, FastText.
- **Avantage** : Simples et rapides à utiliser.
- **Limite** : Ne distinguent pas les mots à significations multiples (e.g., "bank" = banque/rive).

---

### **Contextual Embeddings (Représentations contextuelles)** :
- **Vecteur dynamique** : Le vecteur d’un mot change en fonction du contexte.
- **Exemples** : BERT, GPT, ELMo.
- **Avantage** : Capturent les différentes significations d’un mot selon son contexte.
- **Limite** : Plus coûteux en calcul.

---

### **Résumé des usages** :
- **Statique** : Tâches simples où le contexte importe peu.
- **Contextuel** : Tâches complexes nécessitant une compréhension fine du langage (traduction, résolution d’ambiguïté).

## 22. What are dense and sparse embeddings? Provide examples.


### **Différence entre Sparse et Dense Embeddings :**

1. **Sparse Embeddings (Représentation Creuse) :**
   - **Caractéristique :** Grande dimension avec principalement des zéros.
   - **Exemple :** One-hot encoding pour "chien" → `[0, 1, 0, 0]`.
   - **Limites :** 
     - Pas de sens sémantique.
     - Inefficace en mémoire et calcul.

2. **Dense Embeddings (Représentation Dense) :**
   - **Caractéristique :** Faible dimension avec des valeurs riches et non nulles.
   - **Exemple :** Word2Vec pour "chien" → `[0.8, 0.5, -0.3]`.
   - **Avantages :** 
     - Capturent des relations sémantiques (proximité entre "chien" et "chat").
     - Compactes et efficaces.

---

### **Résumé :**
- Sparse embeddings sont simples mais inefficaces et sans relations sémantiques.
- Dense embeddings sont compacts, efficaces et capturent le sens et les similitudes, largement utilisés dans les modèles modernes comme Word2Vec et BERT.

## 20. What is the difference between Glove and Word2Vec ?

**Word2Vec utilise une fenêtre de contexte (généralement petite) pour apprendre des relations entre les mots. Cela permet de capturer des associations locales entre les mots qui apparaissent fréquemment dans un même contexte.**
**Limite : Ce processus néglige les relations globales et les co-occurrences entre mots dans l'ensemble du corpus.**
**---> GloVe : Capture des relations globales en utilisant la matrice de co-occurrence sur l'ensemble du corpus.**

## 21. What is negative sampling and why is it needed? What other tricks for Word2Vec do you know, and how can you apply them?

**Negative Sampling : Accélère l'entraînement en mettant à jour un sous-ensemble de mots négatifs au lieu de tous les mots.**

## 23. Why might the dimensionality of embeddings be important?

**La dimensionnalité des embeddings doit être choisie en fonction de la complexité des données, de la taille du vocabulaire et des besoins des modèles aval. Un bon choix équilibre précision, efficacité et généralisation.**

## 24. What problems can arise when training Word2Vec on short textual data, and how can you deal with them?

**Lorsque vous entraînez Word2Vec sur des données courtes, vous devez gérer la limitation du vocabulaire, le risque de sur-ajustement et la faible richesse du contexte. Utiliser des techniques d'enrichissement de données, de régularisation et de pré-traitement peut aider à améliorer les performances du modèle.**